In [ ]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import time
import re
from selenium import webdriver
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize,sent_tokenize
import collections
from tika import parser
import re
import xlwt
from xlwt import Workbook
import os
import itertools

all_contents = []
#link_url =[]
contents = []
#url_desc_list = []
#url_pair_list = []
with open(r'/Users/rajsharavan/Desktop/Python/url_state_street.csv','r') as csvf: # Open file in read mode
    urls = csv.reader(csvf)
    for url in urls:
        contents.append(url[0])
        
row=1
wb = Workbook()
add = wb.add_sheet('count_sheet_6', cell_overwrite_ok=True)
    
col=5
j=0
add.write(0,0, 'Job ID')
add.write(0,1, 'Instituion')
add.write(0,2, 'List ID')
add.write(0,3, 'URL')
add.write(0,4, 'List ID')


for joburl in contents[0:500]:
    #print(joburl)
    browser = webdriver.Chrome()
    browser.get(joburl)
    # Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
    #        print('lastCount and lenOfPage matched: ',lenOfPage)
            match=True

    # Now that the page is fully scrolled, grab the source code.
    source_data = browser.page_source
    
    # Throw your source into BeautifulSoup and start parsing!
    bs_data = bs(source_data, "lxml")

    job_descrp = bs_data.find_all('div',attrs={'class':'GWTCKEditor-Disabled'}) #extracts all the text in this tag

    jobDescList = []

    for job in job_descrp:
        jobDescList.append(job.get_text())

    # print(len(jobDescList))
    # print(jobDescList[1])
    browser.close()

    string = ""

    for i in range(0,(len(jobDescList)-1)):
        string = string+jobDescList[i]

    all_contents.append(string)
    


    def get_count(word_count_tuple):
        return word_count_tuple[1]

    word_counts = collections.Counter(all_contents)

    # This function sorts the words based on the number of times repeated and saves them in a csv file
    def print_top():
        word_count = word_counts
        items = sorted(word_count.items(),key=get_count,reverse=True)
        with open(r'/Users/rajsharavan/Desktop/Python/Test/jobdesc_SS.csv','w') as out:
            csv_out=csv.writer(out)
            csv_out.writerow('')
            #csv_out.writerow('Name')
            for row in items:
                csv_out.writerow(row)
    print_top()


    stop_words = set(stopwords.words('english'))


    i=0
    for data in all_contents:
        word_tokens1 = word_tokenize(data)
        words1 = [w for w in word_tokens1 if w.isalpha()]
        #print(type(words1))
        filtered_sentence = []
        i = i+1
#        print("DATA", i)
        for w in words1: 
            w.lower()
            #print(w.lower())
            if w.lower() not in stop_words: 
                filtered_sentence.append(w)
                #print('Total Words :',len(filtered_sentence))
        #print(filtered_sentence)
    
        word_counts = collections.Counter(filtered_sentence)

    # It sorts the words xbased on the number of times repeated and saves them in a csv file
    def print_top1():
        word_count = word_counts
        items = sorted(word_count.items(),key=get_count,reverse=True)
        with open(r'/Users/rajsharavan/Desktop/Python/Test/wordss.csv','w') as out:
            csv_out=csv.writer(out)
            csv_out.writerow('')
            for row in items:
                csv_out.writerow(row)

    print_top1()

    filtered_sentence_lowercase = [x.lower() for x in filtered_sentence]



    contents1 = []
    with open(r'/Users/rajsharavan/Desktop/Python/Test/WordList.csv','r') as csvf: # Open file in read mode
        urls = csv.reader(csvf)
        for url in urls:
            contents1.append(url[0])
        
    contents2 = []
    with open(r'/Users/rajsharavan/Desktop/Python/Test/tfidf_WordList.csv','r') as csvf1: # Open file in read mode
        urls1 = csv.reader(csvf1)
        for url1 in urls1:
            contents2.append(url1[0])
        
    contents3 = []
    with open(r'/Users/rajsharavan/Desktop/Python/Test/Words_TR.csv','r') as csvf2: # Open file in read mode
        urls2 = csv.reader(csvf2)
        for url2 in urls2:
            contents3.append(url2[0])

    list_of_contents = [contents1,contents2,contents3]

    
    for i in range(1, 101):
        add.write(0,i+4,i)
    list_no=1
    for words in list_of_contents:
        for num in words:
            count = 0
            for num1 in filtered_sentence_lowercase:
                if num == num1:
                    count = count+1
            add.write(row,1, 'State Street')
            add.write(row,2, '1')
            add.write(row,3,joburl)
            add.write(row,4, list_no)
            add.write(row,col,count)
            
            col = col+1
            if col == 105:
                col = 5
                row = row +1    
        list_no = list_no+1
    del all_contents[:]            
    del filtered_sentence[:]
    del filtered_sentence_lowercase[:]
    
wb.save('/Users/rajsharavan/Desktop/Python/Test/word_example.csv')